In [48]:
import os
import sys
kernel_name = os.path.basename(sys.executable.replace("/bin/python",""))

if kernel_name == 'py38cu11':
    import ctypes
    ctypes.cdll.LoadLibrary("/apps/gilbreth/cuda-toolkit/cuda-11.2.0/lib64/libcusparse.so.11");
    ctypes.cdll.LoadLibrary("/apps/gilbreth/cuda-toolkit/cuda-11.2.0/lib64/libcublas.so.11");

In [49]:
import DeviceDir

DIR, RESULTS_DIR = DeviceDir.get_directory()
device, NUM_PROCESSORS = DeviceDir.get_device()

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A10
cuda
Cpu count:  32


In [50]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import torch
from tqdm import tqdm
import sklearn.metrics
import DeviceDir
from ipynb.fs.full.DglDataset import get_dataset, train_val_test_mask, generate_synthetic, get_pyg_as_dgl

In [51]:
LOG_INFO = True

In [52]:
DATASET_NAME = 'squirrel'
dataset, graph = get_dataset(DIR, DATASET_NAME , log = True)
# dataset, graph = get_pyg_as_dgl(DATASET_NAME='squirrel', DIR=DIR)


print(graph.num_nodes())
print(graph.num_edges())
print(dataset.num_classes)
print(graph.edges())

num_features = graph.ndata['feat'].shape[1]
num_classes = dataset.num_classes
print(num_features,num_classes)

graph

Done loading data from cached files.
Class:  5
Feature:  torch.Size([5201, 2089])
Train Mask:  torch.Size([5201, 10])
Val Mask:  torch.Size([5201, 10])
Test Mask:  torch.Size([5201, 10])
Label Mask:  torch.Size([5201])
5201
217073
5
(tensor([2849, 3106,  808,  ..., 4785, 5080, 2331]), tensor([3475, 3475, 3475,  ..., 3247, 3247, 3247]))
2089 5


Graph(num_nodes=5201, num_edges=217073,
      ndata_schemes={'feat': Scheme(shape=(2089,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(10,), dtype=torch.bool), 'val_mask': Scheme(shape=(10,), dtype=torch.bool), 'test_mask': Scheme(shape=(10,), dtype=torch.bool)}
      edata_schemes={})

In [53]:
# generate_synthetic(graph, dataset, d=42, h=0.0, train=None, random_state=1, log=True)
# graph

In [54]:
if graph.ndata['train_mask'].ndim>1:
    split_no =0
    train_nids = torch.where(graph.ndata['train_mask'][:,split_no]==True)[0] ##selecting first spilit
    val_nids = torch.where(graph.ndata['val_mask'][:,split_no]==True)[0]
    test_nids = torch.where(graph.ndata['test_mask'][:,split_no]==True)[0]
else:
    train_nids = torch.where(graph.ndata['train_mask']==True)[0]
    val_nids = torch.where(graph.ndata['val_mask']==True)[0]
    test_nids = torch.where(graph.ndata['test_mask']==True)[0]
    
print(train_nids.shape)
print(val_nids.shape)
print(test_nids.shape)

torch.Size([2496])
torch.Size([1664])
torch.Size([1041])


In [55]:
# class NeighborSampler(dgl.dataloading.Sampler):
#     def __init__(self, fanouts : list[int]):
#         super().__init__()
#         self.fanouts = fanouts

#     def sample(self, g, seed_nodes):
#         output_nodes = seed_nodes
#         subgs = []
#         for fanout in reversed(self.fanouts):
#             # Sample a fixed number of neighbors of the current seed nodes.
#             sg = g.sample_neighbors(seed_nodes, fanout)
#             # Convert this subgraph to a message flow graph.
#             sg = dgl.to_block(sg, seed_nodes)
#             seed_nodes = sg.srcdata[NID]
#             subgs.insert(0, sg)
#             input_nodes = seed_nodes
#         return input_nodes, output_nodes, subgs
    

In [56]:
# https://github.com/dmlc/dgl/issues/4512

In [57]:
sampler = dgl.dataloading.NeighborSampler([4, 4])
# sampler = dgl.dataloading.MultiLayerFullNeighborSampler(4)

train_dataloader = dgl.dataloading.DataLoader(
    # The following arguments are specific to DGL's DataLoader.
    graph,  # The graph
    train_nids,  # The node IDs to iterate over in minibatches
    sampler,  # The neighbor sampler
    device=device,  # Put the sampled MFGs on CPU or GPU
    # The following arguments are inherited from PyTorch DataLoader.
    batch_size=1024,  # Batch size
    shuffle=True,  # Whether to shuffle the nodes for every epoch
    drop_last=False,  # Whether to drop the last incomplete batch
    num_workers=0,  # Number of sampler processes
)

In [58]:
input_nodes, output_nodes, mfgs = example_minibatch = next(iter(train_dataloader))

print(example_minibatch)

print(
    "To compute {} nodes' outputs, we need {} nodes' input features".format(
        len(output_nodes), len(input_nodes)
    )
)

print(len(output_nodes))

[tensor([ 651, 2502, 2962,  ..., 4639, 4400, 2022], device='cuda:0'), tensor([ 651, 2502, 2962,  ..., 2424, 2794, 4744], device='cuda:0'), [Block(num_src_nodes=2374, num_dst_nodes=2040, num_edges=7466), Block(num_src_nodes=2040, num_dst_nodes=1024, num_edges=3766)]]
To compute 1024 nodes' outputs, we need 2374 nodes' input features
1024


In [59]:
# print(mfgs[0].srcdata)
# print(mfgs[0].dstdata)

In [60]:
# mfg_1_dst = mfgs[1].dstdata[dgl.NID]
# print(mfg_1_dst)
# print(len(mfg_1_dst))

# #print(len(torch.unique(mfg_1_dst)))

# print(len(output_nodes))

# # print(torch.equal(mfg_0_src[: mfgs[0].num_dst_nodes()], mfg_0_dst))

# # output_nodes = torch.unique(output_nodes)
# # print(len(output_nodes))

# # print(torch.equal(mfg_0_src[: mfgs[0].num_dst_nodes()], mfg_0_dst))

In [61]:
# mfgs[1]
# mfgs[1].num_dst_nodes = lambda ntype=None: len(output_nodes)
# mfgs[1]

In [62]:
mfg_0_src = mfgs[0].srcdata[dgl.NID]
mfg_0_dst = mfgs[0].dstdata[dgl.NID]
print(mfg_0_src)
print(mfg_0_dst)
print(torch.equal(mfg_0_src[: mfgs[0].num_dst_nodes()], mfg_0_dst))

tensor([ 651, 2502, 2962,  ..., 4639, 4400, 2022], device='cuda:0')
tensor([ 651, 2502, 2962,  ..., 4439,  993, 4007], device='cuda:0')
True


In [63]:
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import SAGEConv


class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, aggregator_type="mean")
        self.conv2 = SAGEConv(h_feats, num_classes, aggregator_type="mean")
        self.h_feats = h_feats

    def forward(self, mfgs, x):
        # Lines that are changed are marked with an arrow: "<---"
        
        h_dst = x[: mfgs[0].num_dst_nodes()]  # <---
        h = self.conv1(mfgs[0], (x, h_dst))  # <---
        h = F.relu(h)
        
        h_dst = h[: mfgs[1].num_dst_nodes()]  # <---
        h = self.conv2(mfgs[1], (h, h_dst))  # <---
        
        return h


model = Model(num_features, 128, num_classes).to(device)

In [64]:
def test(loader, N):
    
    model.eval()

    predictions = []
    labels = []
    
    if LOG_INFO:
        pbar = tqdm(total=N)
        pbar.set_description(f'Evaluating')
    
    for input_nodes, output_nodes, mfgs in loader:
        inputs = mfgs[0].srcdata["feat"]
        labels.append(mfgs[-1].dstdata["label"].cpu().numpy())
        predictions.append(model(mfgs, inputs).argmax(1).cpu().numpy())
        
        if LOG_INFO:
            pbar.update(len(output_nodes))

    if LOG_INFO:
        pbar.close()

        

    predictions = np.concatenate(predictions)
    labels = np.concatenate(labels)
    accuracy = sklearn.metrics.accuracy_score(labels, predictions)
    
    return accuracy

In [65]:
def train(epochs = 10):
    opt = torch.optim.Adam(model.parameters())
    
    val_dataloader = dgl.dataloading.DataLoader(graph,val_nids,sampler,batch_size=1024,shuffle=False,drop_last=False,num_workers=0,device=device,)
    test_dataloader = dgl.dataloading.DataLoader(graph,test_nids,sampler,batch_size=1024,shuffle=False,drop_last=False,num_workers=0,device=device,)
    
    best_accuracy = 0
    best_model_path = DIR+DATASET_NAME+"-model.pt"
    
    for epoch in range(epochs):
        model.train()
        
        total_acc = 0; total_loss = 0; total_examples = 0
        
        if LOG_INFO:
            pbar = tqdm(total=len(train_nids))
            pbar.set_description(f'Epoch {epoch}')
    
        for input_nodes, output_nodes, mfgs in train_dataloader:
            # feature copy from CPU to GPU takes place here
            inputs = mfgs[0].srcdata["feat"]
            labels = mfgs[-1].dstdata["label"]
            
            opt.zero_grad()
            predictions = model(mfgs, inputs)

            loss = F.cross_entropy(predictions, labels)            
            loss.backward()
            opt.step()            

            accuracy = sklearn.metrics.accuracy_score(
                labels.cpu().numpy(),
                predictions.argmax(1).detach().cpu().numpy(),
            )
            
            if LOG_INFO:
                pbar.set_postfix(
                    {"loss": "%.03f" % loss.item(), "acc": "%.03f" % accuracy},
                    refresh=False,
                )    

            total_loss += loss.item()
            total_acc += accuracy* len(output_nodes)
            total_examples += len(output_nodes)
            
            if LOG_INFO:
                    pbar.update(len(output_nodes))

        if LOG_INFO:
            pbar.close()

        avg_loss = total_loss/total_examples

        train_accuracy = total_acc/total_examples            
        val_accuracy = test(val_dataloader, len(val_nids))
        test_accuracy = test(test_dataloader, len(test_nids))

        print(f'Epoch {epoch} Loss {avg_loss:0.6f}, Train Acc {train_accuracy:0.4f}, Val Acc {val_accuracy:0.4f}, Test Acc {test_accuracy:0.4f}')

        if best_accuracy < val_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), best_model_path)
    
    print("Best Validation Accuracy: ",best_accuracy)
    
    test_accuracy = test(test_dataloader, len(test_nids))    
    print("Test Accuracy: ",best_accuracy)
    
    return test_accuracy

In [66]:
# train_val_test_mask(graph, train=0.6, val=0.2, test=0.2, random_state=0)

In [67]:
LOG_INFO = True
acc = train(150)
acc

Evaluating: 100%|██████████| 1041/1041 [00:00<00:00, 107282.01it/s]


Epoch 0 Loss 0.001939, Train Acc 0.2188, Val Acc 0.2668, Test Acc 0.2786


Evaluating: 100%|██████████| 1041/1041 [00:00<00:00, 99364.40it/s]

Epoch 1 Loss 0.001775, Train Acc 0.3542, Val Acc 0.3480, Test Acc 0.3506



Evaluating: 100%|██████████| 1041/1041 [00:00<00:00, 112416.85it/s]


...

Evaluating: 100%|██████████| 1041/1041 [00:00<00:00, 93602.39it/s]


Epoch 149 Loss 0.000064, Train Acc 0.9932, Val Acc 0.4219, Test Acc 0.4121
Best Validation Accuracy:  0.49278846153846156


Evaluating: 100%|██████████| 1041/1041 [00:00<00:00, 98724.09it/s]

Test Accuracy:  0.49278846153846156


0.41882804995196926